<a href="https://colab.research.google.com/github/kenhuangsy/hands-on-gradient-boosting/blob/main/02_decision_trees_in_depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [4]:
df_bikes  = pd.read_csv("https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Gradient-Boosting-with-XGBoost-and-Scikit-learn/master/Chapter02/bike_rentals_cleaned.csv")
df_bikes

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,1.0,1.0,12.0,0.0,4.0,1.0,2,0.254167,0.226642,0.652917,0.350133,2114
727,728,1.0,1.0,12.0,0.0,5.0,1.0,2,0.253333,0.255046,0.590000,0.155471,3095
728,729,1.0,1.0,12.0,0.0,6.0,0.0,2,0.253333,0.242400,0.752917,0.124383,1341
729,730,1.0,1.0,12.0,0.0,0.0,0.0,1,0.255833,0.231700,0.483333,0.350754,1796


In [6]:
X_bikes = df_bikes.drop("cnt", axis = 1)
y_bikes = df_bikes["cnt"]
display(X_bikes)
display(y_bikes)

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900
...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,1.0,1.0,12.0,0.0,4.0,1.0,2,0.254167,0.226642,0.652917,0.350133
727,728,1.0,1.0,12.0,0.0,5.0,1.0,2,0.253333,0.255046,0.590000,0.155471
728,729,1.0,1.0,12.0,0.0,6.0,0.0,2,0.253333,0.242400,0.752917,0.124383
729,730,1.0,1.0,12.0,0.0,0.0,0.0,1,0.255833,0.231700,0.483333,0.350754


0       985
1       801
2      1349
3      1562
4      1600
       ... 
726    2114
727    3095
728    1341
729    1796
730    2729
Name: cnt, Length: 731, dtype: int64

In [9]:
# Import the DecisionTreeRegressor and cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

reg = DecisionTreeRegressor(random_state = 42)

scores = cross_val_score(reg, X_bikes, y_bikes, scoring = 'neg_mean_squared_error', cv =5)

# Compute the RMSE and print the results
rmse = np.sqrt(-scores)
round(rmse.mean())

1213

Hmmm our model seems to be performing worse than linear regression. Could it possibly be overfitting?

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes,
                                                    test_size = 0.2,
                                                    random_state = 42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(584, 12)
(584,)
(147, 12)
(147,)


In [14]:
# Check the error of the training set before it makes prediction on the test set

reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)

from sklearn.metrics import mean_squared_error
reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

0.0

This means that our data is overfitting the training data => high variance. Let's see how we can fix this with hyperparameters...

## Hyperparameters

Our first hyperparameter is **max_depth**: it defines the depth of the tree, determined by the number of times splits are made. By default, there is no limit to max_depth, so there may be hundreds or thousands of splits that result in overfitting. By **limitting max_depth to smaller numbers, variance is reduced, and the model performs better on new data**

How to choose the best number for **max_depth**? We can use GridSearchCV

GridSearchCV searches a grid of hyperparameters using crossvalidation to deliver the best results. You have to establish a dictionary of hyperparameter values. One strategy is to select a smallest and largest value with evenly spaced numbers in between. If we are trying to reduce overfitting for example, we can reduce max_depth by putting numbers more on the lower side

Generally, **decreasing max hyperparameters and increasing min hyperparameters will reduce variance and prevent overfitting**

In [15]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[None,2,3,4,6,8,10,20]} # None is default

# Initialize a DecisionTreeRegressor and place it inside of GridSearchCV with params and scoring
reg = DecisionTreeRegressor(random_state=42)
grid_reg = GridSearchCV(reg,
                        params,
                        scoring = 'neg_mean_squared_error',
                        cv = 5,
                        n_jobs = -1)

grid_reg.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 2, 3, 4, 6, 8, 10, 20]},
             scoring='neg_mean_squared_error')

In [17]:
best_params = grid_reg.best_params_
print("Best params:", best_params)

Best params: {'max_depth': 4}


max_depth of 6 resulted in the best cross-validation score in the training set

In [19]:
# Compute the best score
best_score = np.sqrt(-grid_reg.best_score_)

print(f"Training score: {round(best_score)}")

Training score: 918


In [21]:
# Let's see the test score
best_model = grid_reg.best_estimator_
y_pred = best_model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred)**0.5
round(rmse_test)

907

Our next hyperparameter is **min_samples_leaf**: provides a restriction by increasing the number of samples that a leaf may have. **min_samples_leaf is designed to reduce overfitting**

When there are no restrictions, min_sample_leaf = 1 is the default, meaning that leaves may consist of unique sampels (prone to overfitting). **Increasing min_samples_leaf reduces variance**. Remember that leaf nodes are the ones at the bottom of the tree.

Testing a range of values for min_sample_leaf requires going through the same process

In [22]:
# Let's write a function to display the best parameters, training score, and test score
def grid_search(params: dict, 
                reg = DecisionTreeRegressor(random_state = 42)):
  # Instantiate GridSearchCV as grid_reg
  grid_reg = GridSearchCV(reg, params, scoring = 'neg_mean_squared_error', cv = 5, n_jobs = -1)
  
  # Fit grid_reg on X_train and y_train
  grid_reg.fit(X_train, y_train)

  # Extract best params
  best_params = grid_reg.best_params_

  # Print best params
  print("Best params:", best_params)

  # Compute best score
  best_score = np.sqrt(-grid_reg.best_score_)

  # Print best score
  print(f"Training score: {round(best_score)}")

  # Predict test set labels
  y_pred = grid_reg.predict(X_test)

  # Compute rmse_test
  rmse_test = mean_squared_error(y_test, y_pred)**0.5

  # Print rmse_test
  print(f"Test score: {round(rmse_test)}")

In [23]:
X_train.shape

(584, 12)

Since the training set has 548 rows, this helps determine reasonable values for min_search_leaf. Let's try [1,2,4,6,8,10,20,30] as input of our grid_search

In [24]:
grid_search(params = {'min_samples_leaf': [1,2,4,6,8,10,20,30]})

Best params: {'min_samples_leaf': 10}
Training score: 854
Test score: 910


Let's see what happens if we put min_samples_leaf together with max_depth

In [27]:
grid_search(params = {'max_depth': [None,2,3,4,6,8,10], 'min_samples_leaf': [1,2,4,6,8,10,20,30]})

Best params: {'max_depth': None, 'min_samples_leaf': 10}
Training score: 854
Test score: 910


The result is very surprising. We still have the same min_samples_leaf and no max_depth

In [29]:
# Let's try limiting min_samples_leaf to values greater than three
grid_search(params = {'max_depth': [6,7,8,9,10], 'min_samples_leaf': [3,5,7,9]})

Best params: {'max_depth': 7, 'min_samples_leaf': 7}
Training score: 861
Test score: 894


As we can see, the test score has improved.

### Next move on to the rest of the hyperparameters


**max_leaf_nodes** is similar to min_samples_leaf. Instead of specifying the number of samples per leaf, it specifies the total number of leaves. So, max_leaf_nodes = 10 means that the model cannot have more than 10 leaves.

**max_features** is an effective hyperparameter for reducing variance. Instead of considering every possible feature for a split, it chooses from a select number of features each round.
It's standard to see **max_features** with the following options:
* 'auto' is default, which provides no limitations
* 'sqrt' is the square root of the total number of features
* 'log2' is the log of the total number of features in base 2 => 32 columns resolves to 5 since 2^5 = 32

**min_samples_split** provides a limit to the number of samples required before a split can be made. The default is 2, since two samples may be split into one sample each, ending as simple leaves. If the limit is increased to 5, no further splits are permitted for nodes with five samples or fewer. 

**splitter** => two options: **random** and **best**. Splitter tells the model how to select the feature to split each branch. The **best** option, the defualt, selects the feature that results in the greatest gain of information. The **random** option selects the split randomly. **Change to random if you want to prevent overfitting and diversify trees**

**Criterion** for splitting decision tree regressors and classifiers are different. The **criterion** calculates a number for a possible split and compares it to other options. The split with the best score wins. The options for decision tree regressors are **mse** and **friedman_mse** and **mae**. The default is **mse**. For classifiers, **gini** and **entropy** yield similar results

**min_impurity_decrease** results in a split when the impurity is greater than or equal to this value. Impurity is a measure of how pure the predictions are for every node. A tree with 100% accuracy would have an impurity of 0.0 while a tree with 80% accuracy would have an impurity of 0.20


**min_weight_fraction_leaf** is the minimum weighted fraction of the total weights required to be a leaf. It prevents overfitting and reduces variance. 

## Putting it all together

In general, **max_depth, max_features, min_samples_leaf, max_leaf_nodes, min_impurity_decrease, and min_samples_split** are enough 😀

### Predicting heart disease

In [30]:
df_heart = pd.read_csv("https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Gradient-Boosting-with-XGBoost-and-Scikit-learn/master/Chapter02/heart_disease.csv")
df_heart

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [31]:
# Split into X and y
X = df_heart.drop("target", axis = 1)
y = df_heart["target"]
display(X)
display(y)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

In [33]:
# Split into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(242, 13)
(242,)
(61, 13)
(61,)


In [34]:
# Before implementing hyperparameters, it's useful to have a baseline model for comparison
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state = 42)
scores = cross_val_score(model, X, y, cv = 5)
print(f"Accuracy: {np.round(scores, 2)}")
print(f"Accuracy mean: {np.round(scores.mean(), 2)}")

Accuracy: [0.75 0.85 0.75 0.7  0.72]
Accuracy mean: 0.76


The initial accuracy is 76%. Let's see what we can do with hyperparameter fine-tuning

When fine-tuning a lot of hyperparameters, GridSearchCV can be very slow. The sklearn library provides **RandomizedSearchCV** as a wonderful alternative. It tries a random number of combinations. It's not meant to be exhaustive. It's meant to find the best combination in limited time.

In [38]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
# Let's define a function so that we don't have to reproduce the same code over and over again
# params = the hyperparameters
# runs = the number of different combinations our regressor will test
def randomized_search_clf(params, runs = 20, clf = DecisionTreeClassifier(random_state = 42)):
  # Instantiate RandomizedSearchCV as rand_clf
  rand_clf = RandomizedSearchCV(clf, params, n_iter = runs, cv = 5, n_jobs = -1, random_state = 42)
  
  # Fit rand_clf on X_train and y_train
  rand_clf.fit(X_train, y_train)

  # Extract best estimator
  best_model = rand_clf.best_estimator_

  # Extract best score
  best_score = rand_clf.best_score_

  print("Training score:", np.round(best_score, 2))

  # Predict test set labels
  y_pred = best_model.predict(X_test)

  # Compute accuracy
  accuracy = accuracy_score(y_test, y_pred)

  # Print accuracy
  print("Test score:", np.round(accuracy, 2))

  # Return best model
  return best_model




In [39]:
# Let's try to implement some hyperparameters

randomized_search_clf(params={'criterion':['entropy', 'gini'],
                              'splitter':['random', 'best'],
                              'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01],
                              'min_samples_split':[2, 3, 4, 5, 6, 8, 10],
                              'min_samples_leaf':[1, 0.01, 0.02, 0.03, 0.04],
                              'min_impurity_decrease':[0.0, 0.0005, 0.005, 0.05, 0.10, 0.15, 0.2],
                              'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
                              'max_features':['auto', 0.95, 0.90, 0.85, 0.80, 0.75, 0.70],
                              'max_depth':[None, 2,4,6,8],
                              'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01, 0.05]
                            })

Training score: 0.8
Test score: 0.82


DecisionTreeClassifier(criterion='entropy', max_depth=6, max_features=0.95,
                       max_leaf_nodes=50, min_impurity_decrease=0.0005,
                       min_samples_leaf=0.04, min_samples_split=10,
                       min_weight_fraction_leaf=0.0075, random_state=42)

#### Narrowing the range

narrowing the range is one strategy to improve hyperparameters

In [40]:
randomized_search_clf(params={'max_depth':[None, 6, 7],
'max_features':['auto', 0.78],
'max_leaf_nodes':[45, None],
'min_samples_leaf':[1, 0.035, 0.04, 0.045, 0.05],
'min_samples_split':[2, 9, 10],
'min_weight_fraction_leaf': [0.0, 0.05, 0.06, 0.07],
},
runs=100)

Training score: 0.79
Test score: 0.79


DecisionTreeClassifier(max_features='auto', min_samples_split=9,
                       min_weight_fraction_leaf=0.05, random_state=42)

For a proper baseline of comparison, it's essential to put the new model into cross_val_clf. This may be achieved by copying and pasting the preceding model

In [44]:
# Initialize Decision Tree Classifier
model = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=0.78, max_leaf_nodes=45,
            min_impurity_decrease=0.0,
            min_samples_leaf=0.045, min_samples_split=9,
            min_weight_fraction_leaf=0.06, random_state=2,
            splitter='best')

# Obtain scores of cross-validation
scores = cross_val_score(model, X, y, cv=5)

# Display accuracy
print('Accuracy:', np.round(scores, 2))

# Display mean accuracy
print('Accuracy mean: %0.2f' % (scores.mean()))

Accuracy: [0.82 0.9  0.8  0.8  0.78]
Accuracy mean: 0.82


#### feature_importances_

Gives the most important features of a ml model

When testing, it's important not to mix and match training and test sets. After a final model has been selected, however, fitting the model on the entire dataset can be beneficial. Why? Because the goal is to test the model on data that has never been seen and fitting the model on the entire dataset may lead to additional gains in accuracy

In [46]:
best_clf = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=0.78, max_leaf_nodes=45,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=0.045, min_samples_split=9,
                       min_weight_fraction_leaf=0.06,
                       random_state=2, splitter='best')
best_clf.fit(X, y)

DecisionTreeClassifier(max_depth=7, max_features=0.78, max_leaf_nodes=45,
                       min_samples_leaf=0.045, min_samples_split=9,
                       min_weight_fraction_leaf=0.06, random_state=2)

In [47]:
# use feature_importances_
best_clf.feature_importances_

array([0.04826754, 0.04081653, 0.48409586, 0.00568635, 0.        ,
       0.        , 0.        , 0.00859483, 0.        , 0.02690379,
       0.        , 0.18069065, 0.20494446])

In [48]:
# Zip columns and feature_importances_ into dict
feature_dict = dict(zip(X.columns, best_clf.feature_importances_))

# Import operator
import operator

# Sort dict by values (as list of tuples)
sorted(feature_dict.items(), key=operator.itemgetter(1), reverse=True)[0:3]

[('cp', 0.4840958610240171),
 ('thal', 0.20494445570568706),
 ('ca', 0.18069065321397942)]

This tells us that the most important features are:
* 1) 'cp' = chest pain
* 2) 'thalach' = maximum heart rate achieved
* 3) 'ca' = number of major vessals